Tokenize the Block's documents

In [3]:
import re
import nltk
import string

def tokenize(list_of_contexts):
    all_tokens = []         # a list to save all of tokens of all documents of a block
    for i, doc in enumerate(list_of_contexts):
        lower_doc = doc.lower()              # make all of contexts lower case
        list_of_contexts[i] = lower_doc          
        tokens = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?|\w+', list_of_contexts[i])   # tokenize the text with regex
        all_tokens.append(tokens)

    return all_tokens

Removing Stopwords and other useless tokens

In [4]:
def token_filtering(all_tokens):
    for i, doc in enumerate(all_tokens):
        new_tokens = []
        for token in doc:                           # delete all of stopwords and single character tokens except numbers from token list
            if (len(token) < 2 and token.isalpha()) or (token in stop_words):          
                continue
            else:
                new_tokens.append(token)
        all_tokens[i] = new_tokens

    return all_tokens

# Download the stopwords
nltk.download('stopwords')
nltk.download('punkt')
# Get the list of stopwords for English
stop_words = set(nltk.corpus.stopwords.words('english'))

    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Build Term-TermID map

In [5]:
def term_to_termID(term_termID_map, all_tokens, count):
    for doc in all_tokens:
        for token in doc:
            if token not in list(term_termID_map.keys()):
                term_termID_map[token] = count             # add non repeated tokens to term_termID map with a uniqe term ID
                count += 1

    return term_termID_map, count

Gamma encoding and decoding

In [49]:
import math


def gamma_encode_bitwise(docIDs):  # function for gamma encoding the docIDs list
    gaps = [docIDs [0]] + [docIDs [i] - docIDs [i-1] for i in range(1, len(docIDs))]
    gamma = ''            # empty string for the gamma encoded string
    for x in gaps:      
        msb = math.floor(math.log2(x))     # find the highest power of 2 that x contains, which is equivalent to finding the position of the most significant bit of x
        unary = '0' * msb + '1'            # generate a unary code for the msb position, which is a string of msb zeros followed by a one
        binary = bin(x)[3:]         # generate a binary code for x, which is the binary representation of x without the leading one  
        gamma += unary + binary           # concatenate the unary code and the binary code to get the gamma code for x

    return gamma      # return the gamma encoded string


def decode_gamma(gamma):         # function for gamma decoding the docIDs list
    decoded = []       # an empty list for the decoded docIDs
    current = 0      # the current docID
    # split the gamma encoded string into gamma codes for each gap
    # use a loop to read and count the ones until reaching the first zero
    i = 0
    while i < len(gamma):
        count = 0
        while gamma [i] == '0':
            count += 1
            i += 1
    
        binary = '1' + gamma [i+1:i+1+count]        # read the next count bits and prepend a one to them
        gap = int(binary, 2)            # convert the binary code to a decimal number
        current += gap           # add the gap to the current docID to get the next docID
        decoded.append(current)    # append the next docID to the decoded list
        i += 1 + count            # update the index to skip the zero and the binary code

    return decoded


Get the key of a given value

In [51]:
def get_key(val, my_dict):     # return the key for a given value in dictionary  
    for key, value in my_dict.items():
        if val == value:
            return key

Inverted Index's Data Structue

In [79]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.posting_list = ""
        self.is_word = False
        self.term_freq = 0

class Trie:
    def __init__(self):
        self.root = TrieNode()

    # function to insert new terms to the trie tree
    def insert(self, word, docID):
        current_node = self.root
        for char in word:
            # If the character is not in the current node's children, add it as a new child node
            if char not in current_node.children:
                current_node.children[char] = TrieNode()
            current_node = current_node.children[char]
        # Mark the end of the word
        current_node.is_word = True
        current_node.term_freq += 1
        # add document ID to posting list of the added term
        docIDs = decode_gamma(current_node.posting_list)
        if docID not in docIDs:
            docIDs.append(docID)
            sorted(docIDs)
            gamma = gamma_encode_bitwise(docIDs)
            current_node.posting_list = gamma

    # function to check if a term is present in the tree or not
    def search(self, word):
        current_node = self.root
        for char in word:
            # If the character is not in the current node's children, the word is not in the trie
            if char not in current_node.children:
                return False
            current_node = current_node.children[char]
        return current_node.is_word

    # function to return the term's node when it is present in the tree
    def get(self, word):
        current_node = self.root
        for char in word:
            current_node = current_node.children[char]
        # Return the last node itself
        return current_node

    # A function to print all the terms in the trie
    def print_words(self, node=None, prefix=""):
        if node is None:
            node = self.root
        # If the node is a word, print the prefix
        if node.is_word:
            print(prefix, node.term_freq , decode_gamma(node.posting_list))
        # Loop through each child node
        for char, child in node.children.items():
            # Recursively call the function with the child node and the updated prefix
            self.print_words(child, prefix + char)


Get the address and names of all documents

In [80]:
import os 

# where the folder of all docs is
path = "C:\\Users\\ASC\\OneDrive\\Desktop\\temp\\documents" 
os.chdir(path) 
name_of_docs = []  # name of documents
list_of_docs_address = []  # list of address of documents

for file in os.listdir():                # save the file passes in a list
    file_path = f"{path}\{file}"
    list_of_docs_address.append(file_path)
    name_of_docs.append(file[:-4])


print("All of the documents are: ")
for name in name_of_docs:
    print(name, end=" - ")

All of the documents are: 
A Festival of Books - A Murder-Suicide - Better To Be Unlucky - Cloning Pets - Crazy Housing Prices - Food Fight Erupted in Prison - Freeway Chase Ends at Newsstand - Gasoline Prices Hit Record High - Happy and Unhappy Renters - Jerry Decided To Buy a Gun - Man Injured at Fast Food Place - Pulling Out Nine Tons of Trash - Rentals at the Oceanside Community - Sara Went Shopping - Trees Are a Threat - 

Read each block's documents and preprocess them, and build the (term, docID) pairs and sort them for each block

In [81]:
def block_read(doc_per_block):
    my_DISC = []                      # a simulation of computer's disc
    blocks = []                       # save all blocks
    term_termID_map = {}              # map each term with a term ID
    i, count = 0, 0
    while i < len(list_of_docs_address):   # put each n documents in a block
        block = []
        for j in range(i, i+doc_per_block):
            if j < len(list_of_docs_address):
                block.append(list_of_docs_address[j])
        blocks.append(block)
        i += doc_per_block

    for block_number, block in enumerate(blocks):    # do for each block
        list_of_contexts = []
        termID_docID_pair = []
        for doc in block:                            # do for exh document in a block
            context = "" 
            with open(doc, 'r') as f:  
                context = f.read()
                list_of_contexts.append(context)            # read the context of the document and put it in a list
        all_tokens = tokenize(list_of_contexts)     # tokenize and preprocess the context of each block
        all_tokens = token_filtering(all_tokens)
        term_termID_map, count = term_to_termID(term_termID_map, all_tokens, count)  # update term termID map
        for j, doc in enumerate(all_tokens):
            docID = (block_number * doc_per_block) + j
            for token in doc:
                termID = term_termID_map[token]
                termID_docID_pair.append((termID, docID))       # pair each term with document ID in a touple and put it in a list for later proccessing

        termID_docID_pair = sorted(termID_docID_pair)   # sort pairs for each block
        my_DISC.append(termID_docID_pair)   # save the result of each block in dics
    
    return my_DISC, term_termID_map


    

my_disc, term_termID_map = block_read(3)


Merge the result of each block and build the final inverted index

In [82]:
def merge_blocks(my_disc):
    inverted_index = Trie()      # create the inverted index
    heap = []                  # a min heap
    for block_number, block in enumerate(my_disc):
        heap.append((block[0][0], block[0][1], block_number))   # push first element of each block to min heap, insert the block id of them
    end = False
    while not end:       # till all blocks becoe empty
        heap= sorted(heap)       # sort the heap to be a min heap
        term = get_key(heap[0][0], term_termID_map)    # pick the smallest element of heap
        docID = heap[0][1] + 1
        block_number = heap[0][2]
        inverted_index.insert(term, docID)        # insert the min element to the inverted index

        heap.pop(0)                      # pop the inserted element from heap and its block
        my_disc[block_number].pop(0)
        if len(my_disc[block_number]) != 0:      # if its block isn't empty push the next minimum element of the block to heap
            heap.append((my_disc[block_number][0][0], my_disc[block_number][0][1], block_number))
        end = True
        for block in my_disc:    # check if all blocks are empty
            if len(block)!=0:
                end = False
                break

    return inverted_index    

new_dics = my_disc 
inverted_index = merge_blocks(new_dics)

Show the inverted index

In [83]:
inverted_index.print_words()

people 10 [1, 2, 8, 9, 13, 15]
per 2 [8, 13]
percent 9 [1, 5, 6, 8, 9, 13, 15]
period 2 [1, 3]
perfect 1 [4]
permitted 1 [9]
persons 1 [9]
personal 1 [15]
penny 1 [8]
popular 4 [1, 6]
police 11 [2, 7, 8, 9, 12, 13]
portable 1 [12]
post 1 [13]
park 1 [13]
parking 3 [1, 13]
parked 1 [7]
part 1 [13]
party 1 [13]
pain 1 [2]
paid 4 [5, 14, 15]
pay 4 [3, 4, 5, 15]
paying 1 [8]
paper 2 [5, 6]
passing 2 [5, 8]
pasadena 1 [14]
price 5 [4, 5, 8, 14]
prices 7 [1, 5, 8, 15]
prison 3 [6]
prisoners 3 [6]
prince 2 [11]
private 1 [15]
president 1 [4]
pretty 2 [8, 15]
produce 3 [4, 13]
profit 1 [5]
provide 1 [6]
proven 1 [13]
proprietor 1 [7]
proposal 1 [13]
property 2 [15]
problem 2 [11, 13]
problems 4 [13, 15]
probably 1 [11]
proceeds 1 [12]
pros 1 [13]
practice 1 [10]
practiced 1 [9]
place 3 [2, 5, 10]
play 1 [3]
playing 3 [9]
played 1 [9]
player 2 [9]
playground 1 [15]
plates 1 [6]
plastic 1 [12]
plants 1 [15]
planners 1 [15]
plowed 1 [7]
plot 1 [10]
plus 1 [14]
piano 2 [3, 9]
pianist 1 [9]
picked 